<a href="https://colab.research.google.com/github/saeidsaadatigero/combine_question_answers/blob/main/combine_question_answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate==0.21.0

In [ ]:
import json
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch

In [ ]:
with open('/data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [ ]:
dataset = Dataset.from_dict(data)
dataset

Dataset({
    features: ['سوال', 'جواب'],
    num_rows: 5346
})

In [ ]:
def combine_question_answers(examples):
    combined_text = examples["سوال"].join(examples["جواب"])
    return {"text": combined_text}

In [ ]:
dataset = dataset.map(combine_question_answers)

Map:   0%|          | 0/5346 [00:00<?, ? examples/s]

In [ ]:
model_name = 'universitytehran/PersianMind-v1.0'
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['سوال', 'جواب'])
tokenized_datasets

Map:   0%|          | 0/5346 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 5346
})

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=True,
    gradient_accumulation_steps=8
)

In [ ]:
trainer = Trainer(
    model=AutoModelForCausalLM.from_pretrained(model_name),
    args=training_args,
    train_dataset=tokenized_datasets
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]